In [1]:
import pickle
dump = pickle.load(open('mem_obj.dump', 'rb'))

In [2]:
len(dump)

689

In [28]:
sum_size = 0
for obj, count_size in zip(dump.keys(), dump.values()):
    if obj == '':
        continue
    wrong_words = {'dalvik', 'boot', 'font', 'Chrome', 'kgsl', 'system', 'lib', 'Audio', 'System', '.dat', '/dev/__properties__', 'dex', 'apk', 'web', 'config', 'cfi', 'binder', '.bss', 'thread', 'bionic'}
    good = True
    for word in wrong_words:
        if word in obj:
            good = False
            break
    if good:
        print(obj)
        print(count_size)
        print('-' * 80)
        sum_size += count_size[1]
print(sum_size)

/dev/ashmem/shared_memory/D5A74A121842A47E7EAF68382AD3794C
[1, 131072]
--------------------------------------------------------------------------------
/dev/ashmem/shared_memory/1D53F3F180B6B1A832373957D4805C13
[1, 28672]
--------------------------------------------------------------------------------
anon_inode:dmabuf
[3, 12288]
--------------------------------------------------------------------------------
[anon:linker_alloc_vector]
[6, 28672]
--------------------------------------------------------------------------------
[anon:linker_alloc]
[69, 323584]
--------------------------------------------------------------------------------
[anon:atexit
[12, 53248]
--------------------------------------------------------------------------------
[anon:linker_alloc_small_objects]
[14, 65536]
--------------------------------------------------------------------------------
/data/data/com.innogames.riseofcultures/no_backup/androidx.work.workdb-shm
[1, 32768]
-----------------------------------